In [1]:
import numpy
import numpy as np
from numpy.lib.stride_tricks import as_strided
import argparse
import cv2
import time
import math
import copy

In [2]:
imageTable = np.array([[178,228,14,164,246,210,82,79,14,157,230,154,145,204,28,178],
                       [239,104,202,194,64,21,225,182,255,50,100,215,75,177,118,239],
                       [216,155,78,195,38,3,105,101,92,172,179,24,21,43,172,216],
                       [110,99,189,54,187,86,29,167,32,140,91,189,10,46,123,110],
                       [28,172,166,157,34,35,18,118,74,241,119,88,146,92,212,28],
                     [61,97,165,129,222,93,223,56,57,215,155, 242,102,121,249,61],
                      [149,69,22,220,77,146,47,68,232,203, 16,221,150,65,225,149],
                       [135,100,192,110,174,131,41,248,141,124,19,178,180,201,51,135],
                     [251,117,102,31,57,161,146,123,244,161,61,51,41,33,158,251],
                       [47,235,136,252,186,65,145,138,159,228,169,195,66,244,56,47],
                       [165,63,194,127,53,79,3,21,85,8,185,41,212,141,38,165],
                       [20,249,177,38,50,227,156,144,49,41,230,245,49,245,253,20],
                       [195,121,21,95,197,243,116,230,235,4,86,204,191,148,150,195],
                       [224,26,207,78,12,95,189,67,105,72,212,215,245,249,60,224],
                       [5,8,7,98,233,176,124,251,220,129,82,236,222,137,7,5],
                       [178,228,14,164,246,210,82,79,14,157,230,154,145,204,28,178]])
searchTemplate = np.array([[123, 250,185],[170,24,20], [117,102,31]] )

In [3]:
# find the value from the center
def find_mid(input_img):
    find_mid.middleindex=(input_img.shape[0]/2,input_img.shape[1]/2)
    find_mid.middleindex=int(find_mid.middleindex[0]),int(find_mid.middleindex[1])
    find_mid.midpoint=input_img[find_mid.middleindex]
    print(find_mid.middleindex,find_mid.midpoint)
find_mid(imageTable)

(8, 8) 244


**Full search**

In [4]:
#For full search
def fullsearch(input_image, template):
    window_size = template.shape
    y = as_strided(input_image,
                    shape=(input_image.shape[0] - window_size[0] + 1,
                           input_image.shape[1] - window_size[1] + 1,) +window_size,
                           strides=input_image.strides * 2)
    
    # Compute the sum of squared differences using broadcasting.
    ssd = ((y - template) ** 2).sum(axis=-1).sum(axis=-1)
    minpoint=np.where(ssd==numpy.amin(ssd))
    listOfCoordinates = list(zip(minpoint[0], minpoint[1]))
    for cord in listOfCoordinates:
        print("Block that has lowest ssd and the ssdvalue")
        print(y[cord],numpy.amin(ssd))

In [5]:
fullsearch(imageTable, searchTemplate)

Block that has lowest ssd and the ssdvalue
[[164 246 210]
 [194  64  21]
 [195  38   3]] 15463


In [6]:
def midmatrix(midpointindex,radius=0):
    p1=midpointindex[0]-radius,midpointindex[1]+radius
    p2=midpointindex[0],midpointindex[1]+radius
    p3=midpointindex[0]+radius,midpointindex[1]+radius
    p4=midpointindex[0]-radius,midpointindex[1]
    p5=midpointindex
    p6=midpointindex[0]+radius,midpointindex[1]
    p7=midpointindex[0]-radius,midpointindex[1]-radius
    p8=midpointindex[0],midpointindex[1]-4
    p9=midpointindex[0]+radius,midpointindex[1]-radius
    return p1,p2,p3,p4,p5,p6,p7,p8,p9
def creatmatrix(inputmatrix,centerpoint):
    pt1=centerpoint[0]-1,centerpoint[1]+1
    value1=inputmatrix[pt1]
    pt2=centerpoint[0],centerpoint[1]+1
    value2=inputmatrix[pt2]
    pt3=centerpoint[0]+1,centerpoint[1]+1
    value3=inputmatrix[pt3]
    pt4=centerpoint[0]-1,centerpoint[1]
    value4=inputmatrix[pt4]
    pt5=centerpoint
    value5=inputmatrix[pt5]
    pt6=centerpoint[0]+1,centerpoint[1]
    value6=inputmatrix[pt6]
    pt7=centerpoint[0]-1,centerpoint[1]-1
    value7=inputmatrix[pt7]
    pt8=centerpoint[0],centerpoint[1]-1
    value8=inputmatrix[pt8]
    pt9=centerpoint[0]+1,centerpoint[1]-1
    value9=inputmatrix[pt9]
    creatmatrix.matrix=np.array([[value1,value2,value3],[value4,value5,value6],[value7,value8,value9]])
    return creatmatrix.matrix
#Calculate the sum of square different
def ssdcalc(img,template):
    ssdcalc.ssd=0
    height,width=searchTemplate.shape
    for i in range(height):
        for j in range(width):
            diff = img[i][j] - template[i][j]
            ssdcalc.ssd += diff * diff
def stagecalulation(centerpoint,radius):
    stagecalulation.midpointlist=[]
    centerpoint=centerpoint
    stagecalulation.midpointlist=midmatrix(centerpoint,radius=radius)
    stagecalulation.ssdlist=[]
    for i in range(9):
        creatmatrix(imageTable,stagecalulation.midpointlist[i])
        ssdcalc(creatmatrix.matrix,searchTemplate)
        stagecalulation.ssdlist.append(ssdcalc.ssd)

**Three step search**

In [7]:
stage=[1,2]
radius=4
#first stage
midmatrix(find_mid.middleindex,radius=4)
stagecalulation(centerpoint=find_mid.middleindex,radius=4)
minlist_pos=stagecalulation.ssdlist.index(min(stagecalulation.ssdlist))
minlist_value=(min(stagecalulation.ssdlist))
#second and third stage
for i in stage:
    radisus=radius/2
    midmatrix(stagecalulation.midpointlist[minlist_pos],radius=radius)
    stagecalulation(centerpoint=stagecalulation.midpointlist[minlist_pos],radius=radius)
    minlist_pos=stagecalulation.ssdlist.index(min(stagecalulation.ssdlist))
    minlist_value=(min(stagecalulation.ssdlist))
    
#The bestpoint has been found at stage3 at index 4 of 3x3 array
#Best match in the Imagetemplate and the midpoint of that matrix
print('Search result:Best match in the Imagetemplate and the midpoint of the matrix')
print(creatmatrix(imageTable,stagecalulation.midpointlist[minlist_pos]),stagecalulation.midpointlist[minlist_pos])

Search result:Best match in the Imagetemplate and the midpoint of the matrix
[[140 241 215]
 [ 32  74  57]
 [167 118  56]] (4, 8)


**Four step search**

In [8]:
stage=[1,2]
radius=2
#first stage
stagecalulation(centerpoint=find_mid.middleindex,radius=4)
minlist_pos=stagecalulation.ssdlist.index(min(stagecalulation.ssdlist))
minlist_value=(min(stagecalulation.ssdlist))
#second and third stage
for i in stage:
    midmatrix(stagecalulation.midpointlist[minlist_pos],radius=radius)
    stagecalulation(centerpoint=stagecalulation.midpointlist[minlist_pos],radius=radius)
    minlist_pos=stagecalulation.ssdlist.index(min(stagecalulation.ssdlist))
    minlist_value=(min(stagecalulation.ssdlist))

#The bestpoint has been found at stage4 at index 4 of 3x3 array
#Best match in the Imagetemplate and the midpoint of that matrix
print('Search result:Best match in the Imagetemplate and the midpoint of the matrix')
print(creatmatrix(imageTable,stagecalulation.midpointlist[minlist_pos]),stagecalulation.midpointlist[minlist_pos])

Search result:Best match in the Imagetemplate and the midpoint of the matrix
[[140 241 215]
 [ 32  74  57]
 [167 118  56]] (4, 8)


**Diamond Search**

In [9]:
def midmatrix_dimond(midpointindex):
    p1=midpointindex[0],midpointindex[1]+2
    p2=midpointindex[0]-1,midpointindex[1]+1
    p3=midpointindex[0]+1,midpointindex[1]+1
    p4=midpointindex[0]-2,midpointindex[1]
    p5=midpointindex
    p6=midpointindex[0]+2,midpointindex[1]
    p7=midpointindex[0]-1,midpointindex[1]-1
    p8=midpointindex[0]+1,midpointindex[1]-1
    p9=midpointindex[0],midpointindex[1]-2
    return p1,p2,p3,p4,p5,p6,p7,p8,p9

def stagecalulation_dimond(centerpoint):
    stagecalulation_dimond.midpointlist=[]
    centerpoint=centerpoint
    stagecalulation_dimond.midpointlist=midmatrix_dimond(centerpoint)
    stagecalulation_dimond.ssdlist=[]
    for i in range(9):
        creatmatrix(imageTable,stagecalulation_dimond.midpointlist[i])
        ssdcalc(creatmatrix.matrix,searchTemplate)
        stagecalulation_dimond.ssdlist.append(ssdcalc.ssd)
        
#Fourth stage
#Redefind the function as 4th stage has different matrix comp
def midmatrix_dimond_4thstage(midpointindex):
    p1=midpointindex[0],midpointindex[1]+1
    p2=midpointindex[0]-1,midpointindex[1]+1
    p3=midpointindex
    p4=midpointindex[0]+1,midpointindex[1]
    p5=midpointindex[0],midpointindex[1]-1
    return p1,p2,p3,p4,p5
def stagecalulation_dimond_4thstage(centerpoint):
    stagecalulation_dimond_4thstage.midpointlist=[]
    centerpoint=centerpoint
    stagecalulation_dimond_4thstage.midpointlist=midmatrix_dimond_4thstage(centerpoint)
    stagecalulation_dimond_4thstage.ssdlist=[]
    for i in range(5):
        creatmatrix(imageTable,stagecalulation_dimond_4thstage.midpointlist[i])
        ssdcalc(creatmatrix.matrix,searchTemplate)
        stagecalulation_dimond_4thstage.ssdlist.append(ssdcalc.ssd)

In [10]:
dim_stage=[1,2]
#first stage
stagecalulation_dimond(centerpoint=find_mid.middleindex)
minlist_pos_dim=stagecalulation_dimond.ssdlist.index(min(stagecalulation_dimond.ssdlist))
minlist_value_dim=(min(stagecalulation_dimond.ssdlist))
#second and third stage
for i in dim_stage:
    stagecalulation_dimond(centerpoint=stagecalulation_dimond.midpointlist[minlist_pos_dim])
    minlist_pos_dim=stagecalulation_dimond.ssdlist.index(min(stagecalulation_dimond.ssdlist))
    minlist_value_dim=(min(stagecalulation_dimond.ssdlist))
#Fourth stage
stagecalulation_dimond_4thstage(centerpoint=stagecalulation_dimond.midpointlist[minlist_pos_dim])
minlist_pos_dim=stagecalulation_dimond_4thstage.ssdlist.index(min(stagecalulation_dimond_4thstage.ssdlist))
minlist_value_dim=(min(stagecalulation_dimond_4thstage.ssdlist))

##The bestpoint has been found at stage4 at index 2 of 3x3 array
#Best match in the Imagetemplate and the midpoint of that matrix
print('Search result:Best match in the Imagetemplate and the midpoint of the matrix')
print(creatmatrix(imageTable,stagecalulation_dimond_4thstage.midpointlist[minlist_pos_dim]),stagecalulation_dimond_4thstage.midpointlist[minlist_pos_dim])

Search result:Best match in the Imagetemplate and the midpoint of the matrix
[[244 159  85]
 [123 138  21]
 [146 145   3]] (9, 7)


**Hexagon Block Search (HEXBS)**

In [11]:
def midmatrix_HEXBS(midpointindex):
    p1=midpointindex[0]-1,midpointindex[1]+2
    p2=midpointindex[0]+1,midpointindex[1]+2
    p3=midpointindex[0]-2,midpointindex[1]
    p4=midpointindex
    p5=midpointindex[0]+2,midpointindex[1]
    p6=midpointindex[0]-1,midpointindex[1]-2
    p7=midpointindex[0]+1,midpointindex[1]-2
    return p1,p2,p3,p4,p5,p6,p7

def stagecalulation_HEXBS(centerpoint):
    stagecalulation_HEXBS.midpointlist=[]
    centerpoint=centerpoint
    stagecalulation_HEXBS.midpointlist=midmatrix_HEXBS(centerpoint)
    stagecalulation_HEXBS.ssdlist=[]
    for i in range(7):
        creatmatrix(imageTable,stagecalulation_HEXBS.midpointlist[i])
        ssdcalc(creatmatrix.matrix,searchTemplate)
        stagecalulation_HEXBS.ssdlist.append(ssdcalc.ssd)
#Stage 4
#Redefind the function as 4th stage has different matrix comp
def midmatrix_HEXBS_4thstage(midpointindex):
    p1=midpointindex[0],midpointindex[1]+1
    p2=midpointindex[0]-1,midpointindex[1]+1
    p3=midpointindex
    p4=midpointindex[0]+1,midpointindex[1]
    p5=midpointindex[0],midpointindex[1]-1
    return p1,p2,p3,p4,p5
def stagecalulation_HEXBS_4thstage(centerpoint):
    stagecalulation_HEXBS_4thstage.midpointlist=[]
    centerpoint=centerpoint
    stagecalulation_HEXBS_4thstage.midpointlist=midmatrix_HEXBS_4thstage(centerpoint)
    stagecalulation_HEXBS_4thstage.ssdlist=[]
    for i in range(5):
        creatmatrix(imageTable,stagecalulation_HEXBS_4thstage.midpointlist[i])
        ssdcalc(creatmatrix.matrix,searchTemplate)
        stagecalulation_HEXBS_4thstage.ssdlist.append(ssdcalc.ssd)

In [12]:
HEXBS_stage=[1,2]
#first stage
stagecalulation_HEXBS(centerpoint=find_mid.middleindex)
minlist_pos_HEXBS=stagecalulation_HEXBS.ssdlist.index(min(stagecalulation_HEXBS.ssdlist))
minlist_value_HEXBS=(min(stagecalulation_HEXBS.ssdlist))
#second and third stage
for i in HEXBS_stage:
    stagecalulation_HEXBS(centerpoint=stagecalulation_HEXBS.midpointlist[minlist_pos_HEXBS])
    minlist_pos_HEXBS=stagecalulation_HEXBS.ssdlist.index(min(stagecalulation_HEXBS.ssdlist))
    minlist_value_HEXBS=(min(stagecalulation_HEXBS.ssdlist))
#Fourth stage
stagecalulation_HEXBS_4thstage(centerpoint=stagecalulation_HEXBS.midpointlist[minlist_pos_HEXBS])
minlist_pos_HEXBS=stagecalulation_HEXBS_4thstage.ssdlist.index(min(stagecalulation_HEXBS_4thstage.ssdlist))
minlist_value_HEXBS=min(stagecalulation_HEXBS_4thstage.ssdlist)

##The bestpoint has been found at stage4 at index 2 of 3x3 array
#Best match in the Imagetemplate and the midpoint of that matrix
print('Search result:Best match in the Imagetemplate and the midpoint of the matrix')
print(creatmatrix(imageTable,stagecalulation_HEXBS_4thstage.midpointlist[minlist_pos_HEXBS]),stagecalulation_HEXBS_4thstage.midpointlist[minlist_pos_HEXBS])

Search result:Best match in the Imagetemplate and the midpoint of the matrix
[[ 68 248 123]
 [ 47  41 146]
 [146 131 161]] (7, 6)
